In [2]:
import os

training_phish_dir = 'db/htmls/training/Phish/'
training_not_phish_dir = 'db/htmls/training/NotPhish/'
validation_phish_dir = 'db/htmls/validation/Phish/'
validation_not_phish_dir = 'db/htmls/validation/NotPhish/'

training_files_phish = [os.path.join(training_phish_dir, filename) for filename in os.listdir(training_phish_dir)]
training_files_not_phish = [os.path.join(training_not_phish_dir, filename) for filename in os.listdir(training_not_phish_dir)]
validation_files_phish = [os.path.join(validation_phish_dir, filename) for filename in os.listdir(validation_phish_dir)]
validation_files_not_phish = [os.path.join(validation_not_phish_dir, filename) for filename in os.listdir(validation_not_phish_dir)]

# print(training_files_phish)
print(len(training_files_phish))
print(len(training_files_not_phish))
print(len(validation_files_phish))
print(len(validation_files_not_phish))


4216
6139
1055
1535


In [3]:
from bs4 import BeautifulSoup

# Function to preprocess HTML content
def preprocess_html(html):
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text()
    # Add more text preprocessing steps as needed
    return text

# Preprocess the training and validation data
training_data = []
training_labels = []
for file_path in training_files_phish:
    try:
        training_data.append(preprocess_html(open(file_path, 'r').read()))
        training_labels.append('p')
    except:
        continue
for file_path in training_files_not_phish:
    try:
        training_data.append(preprocess_html(open(file_path, 'r').read()))
        training_labels.append('n')
    except:
        continue

validation_data = []
validation_labels = []
for file_path in validation_files_phish:
    try:
        validation_data.append(preprocess_html(open(file_path, 'r').read()))
        validation_labels.append('p')
    except:
        continue
for file_path in validation_files_not_phish:
    try:
        validation_data.append(preprocess_html(open(file_path, 'r').read()))
        validation_labels.append('n')
    except:
        continue
# training_data = [preprocess_html(open(file_path, 'r').read()) for file_path in training_files]
# validation_data = [preprocess_html(open(file_path, 'r').read()) for file_path in validation_files]

# TF-IDF vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train = tfidf_vectorizer.fit_transform(training_data)
X_valid = tfidf_vectorizer.transform(validation_data)


In [6]:
from sklearn.linear_model import LogisticRegression

# Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train, training_labels)


LogisticRegression()

In [7]:
# Make predictions on the validation data
validation_predictions = model.predict(X_valid)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(validation_labels, validation_predictions)
report = classification_report(validation_labels, validation_predictions)

print(f"Accuracy: {accuracy:.2f}")
print(report)


Accuracy: 0.91
              precision    recall  f1-score   support

           n       0.93      0.93      0.93      1535
           p       0.89      0.90      0.89      1055

    accuracy                           0.91      2590
   macro avg       0.91      0.91      0.91      2590
weighted avg       0.91      0.91      0.91      2590



In [5]:
from sklearn.svm import SVC

# Train an SVM model
model_svc = SVC()
model_svc.fit(X_train, training_labels)

SVC()

In [8]:
# Make predictions on the validation data
validation_predictions = model_svc.predict(X_valid)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(validation_labels, validation_predictions)
report = classification_report(validation_labels, validation_predictions)

print(f"Accuracy: {accuracy:.2f}")
print(report)


Accuracy: 0.96
              precision    recall  f1-score   support

           n       0.95      0.97      0.96      1535
           p       0.96      0.93      0.95      1055

    accuracy                           0.96      2590
   macro avg       0.96      0.95      0.95      2590
weighted avg       0.96      0.96      0.96      2590



In [25]:
test = tfidf_vectorizer.transform(["pay"])
pred = model_svc.predict(test)
print(pred)

['p']


In [13]:
import urllib3

In [27]:
# url = 'http://tutorialspoint.com/python/python_overview.htm'
# url = 'https://www.google.com'
# url = 'https://www.apple.com'
# url = 'https://www.apple.com/shop/buy-iphone/iphone-15-pro'
# url = 'https://accounts.google.com'
# url = 'https://www.phishing.org/phishing-examples'
# url = 'https://www.bullionvault.com/help/claim_my_freebie_q.html'
# url = 'https://www.usmoneyreserve.com/resources/free-information-kit/'
# url = 'www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrcmd=_home-customer&nav=1/loading.php'


http = urllib3.PoolManager()

html_doc = http.request('GET', url)

html_data = preprocess_html(html_doc)

test = tfidf_vectorizer.transform([html_data])
pred = model_svc.predict(test)
print(pred)

['n']
